# Product Quality

In [1]:
# 'day-month-year'
start_date = '01-05-17'
end_date = '21-05-17'
city = 'sao paulo'
state = 'SP'

# Products with bad review (disconsidering delays)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import datetime as dt

warnings.filterwarnings('ignore')

start_date =  dt.datetime.strptime(start_date, '%d-%m-%y').date()
end_date =  dt.datetime.strptime(end_date, '%d-%m-%y').date()

orders = pd.read_csv('../dataset/olist_orders_dataset.csv', delimiter = ',')
order_item = pd.read_csv('../dataset/olist_order_items_dataset.csv', delimiter = ',')
products = pd.read_csv('../dataset/olist_products_dataset.csv', delimiter = ',')
reviews = pd.read_csv('../dataset/olist_order_reviews_dataset.csv', delimiter = ',')
customer = pd.read_csv('../dataset/olist_customers_dataset.csv', delimiter = ',')

orders = orders[['order_id','customer_id', 'order_purchase_timestamp', 'order_delivered_customer_date', 'order_estimated_delivery_date']]
order_item = order_item[['order_id','product_id']]
products = products[['product_id','product_category_name']]
reviews = reviews[['order_id','review_score', 'review_creation_date']]
customer = customer[['customer_id', 'customer_city','customer_state']]

final = pd.merge(orders, order_item, on='order_id')
final = pd.merge(final, reviews, on='order_id')
final = pd.merge(final, products, on='product_id')
final = pd.merge(final, customer, on='customer_id')
final['is_delayed'] = final['order_delivered_customer_date'] > final['order_estimated_delivery_date']

reviews = final[['review_score','product_category_name', 'is_delayed']]
insatisfaction = reviews.loc[(reviews['review_score'] <= 3) & (reviews['is_delayed'] == False)]
insatisfaction = insatisfaction.groupby(["product_category_name"]).size().reset_index(name="Bad_reviews")

reviews = reviews.groupby(["product_category_name"]).size().reset_index(name="All_reviews")

insatisfaction = pd.merge(insatisfaction, reviews, on='product_category_name')
insatisfaction['Average_Insatisfaction'] = insatisfaction['Bad_reviews'] / insatisfaction['All_reviews']
insatisfaction = insatisfaction.sort_values('Average_Insatisfaction', ascending=False)
insatisfaction.head(10)

,product_category_name,Bad_reviews,All_reviews,Average_Insatisfaction
42,fraldas_higiene,21,39,0.538462
67,seguros_e_servicos,1,2,0.500000
65,portateis_cozinha_e_preparadores_de_alimentos,7,15,0.466667
55,moveis_escritorio,586,1701,0.344503
60,pc_gamer,3,9,0.333333
5,artigos_de_festas,13,43,0.302326
71,telefonia_fixa,80,265,0.301887
36,fashion_roupa_feminina,15,50,0.300000
38,fashion_roupa_masculina,39,132,0.295455
46,la_cuisine,4,14,0.285714


# Products with good reviews

## General Good Reviews (dased in order day)

In [3]:
satisfaction = final[['review_score','product_category_name', 
                      'review_creation_date', 'order_purchase_timestamp',
                     'customer_city', 'customer_state']]

satisfaction['order_purchase_timestamp'] = pd.to_datetime(satisfaction['order_purchase_timestamp'])
satisfaction['review_creation_date'] = pd.to_datetime(satisfaction['review_creation_date'])

s_date = satisfaction.loc[(satisfaction['order_purchase_timestamp'].dt.date >= start_date)
                    & (satisfaction['order_purchase_timestamp'].dt.date <= end_date)]

s_review_date = satisfaction.loc[(satisfaction['review_creation_date'].dt.date >= start_date)
                    & (satisfaction['review_creation_date'].dt.date <= end_date)]

s_date = s_date.groupby(["product_category_name"]).size().reset_index(name="total")
s_review_date = s_review_date.groupby(["product_category_name"]).size().reset_index(name="total")
s = satisfaction.groupby(["product_category_name"]).size().reset_index(name="total")

order_satisfaction = satisfaction.loc[(satisfaction['review_score'] >=4) 
                                        & (satisfaction['order_purchase_timestamp'].dt.date >= start_date)
                                        & (satisfaction['order_purchase_timestamp'].dt.date <= end_date)]

order_satisfaction_by_city = satisfaction.loc[(satisfaction['review_score'] >=4) 
                                        & (satisfaction['order_purchase_timestamp'].dt.date >= start_date)
                                        & (satisfaction['order_purchase_timestamp'].dt.date <= end_date)
                                        & (satisfaction['customer_city'] == city)]

order_satisfaction_just_by_city = satisfaction.loc[(satisfaction['review_score'] >=4)
                                        & (satisfaction['customer_city'] == city)]

order_satisfaction_by_state = satisfaction.loc[(satisfaction['review_score'] >=4) 
                                        & (satisfaction['order_purchase_timestamp'].dt.date >= start_date)
                                        & (satisfaction['order_purchase_timestamp'].dt.date <= end_date)
                                        & (satisfaction['customer_state'] == state)]
                                              
order_satisfaction_just_by_state = satisfaction.loc[(satisfaction['review_score'] >=4)
                                        & (satisfaction['customer_state'] == state)]

review_satisfaction = satisfaction.loc[(satisfaction['review_score'] >=4) 
                                        & (satisfaction['review_creation_date'].dt.date >= start_date)
                                        & (satisfaction['review_creation_date'].dt.date <= end_date)]

review_satisfaction_by_city = satisfaction.loc[(satisfaction['review_score'] >=4) 
                                        & (satisfaction['review_creation_date'].dt.date >= start_date)
                                        & (satisfaction['review_creation_date'].dt.date <= end_date)
                                        & (satisfaction['customer_city'] == city)]

                                             
review_satisfaction_by_state = satisfaction.loc[(satisfaction['review_score'] >=4) 
                                        & (satisfaction['review_creation_date'].dt.date >= start_date)
                                        & (satisfaction['review_creation_date'].dt.date <= end_date)
                                        & (satisfaction['customer_state'] == state)]

order_satisfaction = order_satisfaction.groupby(["product_category_name"]).size().reset_index(name="amount")
order_satisfaction = order_satisfaction.merge(s_date, on='product_category_name')
order_satisfaction['average_satisfaction'] = order_satisfaction['amount']/order_satisfaction['total']
order_satisfaction = order_satisfaction.sort_values('average_satisfaction', ascending=False)
order_satisfaction.head(10)

,product_category_name,amount,total,average_satisfaction
0,agro_industria_e_comercio,3,3,1.0
13,cine_foto,1,1,1.0
49,sinalizacao_e_seguranca,1,1,1.0
36,livros_tecnicos,3,3,1.0
31,fashion_underwear_e_moda_praia,6,6,1.0
29,fashion_roupa_infanto_juvenil,1,1,1.0
19,dvds_blu_ray,3,3,1.0
16,construcao_ferramentas_ferramentas,2,2,1.0
27,fashion_esporte,3,3,1.0
4,artes_e_artesanato,1,1,1.0


In [4]:
order_satisfaction = order_satisfaction.sort_values('amount', ascending=False)
order_satisfaction.head(10)

,product_category_name,amount,total,average_satisfaction
24,esporte_lazer,181,226,0.800885
33,informatica_acessorios,181,224,0.808036
10,cama_mesa_banho,170,238,0.714286
53,utilidades_domesticas,166,207,0.801932
8,beleza_saude,149,197,0.756345
41,moveis_decoracao,142,208,0.682692
18,cool_stuff,134,173,0.774566
51,telefonia,112,152,0.736842
9,brinquedos,111,129,0.860465
32,ferramentas_jardim,85,104,0.817308


## Order Satisfaction by City

In [5]:
order_satisfaction_by_city = order_satisfaction_by_city.groupby(["product_category_name"]).size().reset_index(name="amount")
order_satisfaction_by_city = order_satisfaction_by_city.merge(s_date, on='product_category_name')
order_satisfaction_by_city['average_satisfaction'] = order_satisfaction_by_city['amount']/order_satisfaction_by_city['total']
order_satisfaction_by_city = order_satisfaction_by_city.sort_values('average_satisfaction', ascending=False)
order_satisfaction_by_city.head(10)

,product_category_name,amount,total,average_satisfaction
23,fashion_underwear_e_moda_praia,3,6,0.500000
22,fashion_roupa_masculina,2,4,0.500000
11,construcao_ferramentas_ferramentas,1,2,0.500000
31,moveis_quarto,3,7,0.428571
21,fashion_roupa_feminina,2,5,0.400000
2,alimentos_bebidas,5,14,0.357143
0,agro_industria_e_comercio,1,3,0.333333
1,alimentos,1,3,0.333333
13,dvds_blu_ray,1,3,0.333333
17,eletroportateis,3,10,0.300000


In [6]:
order_satisfaction_by_city = order_satisfaction_by_city.sort_values('amount', ascending=False)
order_satisfaction_by_city.head(10)

,product_category_name,amount,total,average_satisfaction
9,cama_mesa_banho,32,238,0.134454
18,esporte_lazer,24,226,0.106195
7,beleza_saude,23,197,0.116751
39,utilidades_domesticas,23,207,0.111111
29,moveis_decoracao,21,208,0.100962
25,informatica_acessorios,18,224,0.080357
37,telefonia,18,152,0.118421
8,brinquedos,17,129,0.131783
12,cool_stuff,13,173,0.075145
6,bebes,11,76,0.144737


## Best category by State

In [7]:
order_satisfaction_by_state = order_satisfaction_by_state.groupby(["product_category_name"]).size().reset_index(name="amount")
order_satisfaction_by_state = order_satisfaction_by_state.merge(s_date, on='product_category_name')
order_satisfaction_by_state['average_satisfaction'] = order_satisfaction_by_state['amount']/order_satisfaction_by_state['total']
order_satisfaction_by_state = order_satisfaction_by_state.sort_values('average_satisfaction', ascending=False)
order_satisfaction_by_state.head(10)

,product_category_name,amount,total,average_satisfaction
14,construcao_ferramentas_ferramentas,2,2,1.000000
1,alimentos,2,3,0.666667
24,fashion_roupa_feminina,3,5,0.600000
23,fashion_calcados,7,14,0.500000
26,fashion_underwear_e_moda_praia,3,6,0.500000
43,tablets_impressao_imagem,2,4,0.500000
18,eletrodomesticos_2,4,8,0.500000
25,fashion_roupa_masculina,2,4,0.500000
2,alimentos_bebidas,6,14,0.428571
37,moveis_quarto,3,7,0.428571


In [8]:
order_satisfaction_by_state = order_satisfaction_by_state.sort_values('amount', ascending=False)
order_satisfaction_by_state.head(10)

,product_category_name,amount,total,average_satisfaction
9,cama_mesa_banho,77,238,0.323529
46,utilidades_domesticas,69,207,0.333333
28,informatica_acessorios,64,224,0.285714
7,beleza_saude,62,197,0.314721
21,esporte_lazer,61,226,0.269912
35,moveis_decoracao,53,208,0.254808
15,cool_stuff,47,173,0.271676
44,telefonia,42,152,0.276316
8,brinquedos,40,129,0.310078
27,ferramentas_jardim,28,104,0.269231


## Using review day as parameter

## General Good Reviews

In [9]:
review_satisfaction = review_satisfaction.groupby(["product_category_name"]).size().reset_index(name="amount")
review_satisfaction = review_satisfaction.merge(s_review_date, on='product_category_name')
review_satisfaction['average_satisfaction'] = review_satisfaction['amount']/review_satisfaction['total']
review_satisfaction = review_satisfaction.sort_values('average_satisfaction', ascending=False)
review_satisfaction.head(10)

,product_category_name,amount,total,average_satisfaction
0,agro_industria_e_comercio,1,1,1.0
45,musica,1,1,1.0
39,moveis_colchao_e_estofado,1,1,1.0
21,eletrodomesticos,20,20,1.0
20,dvds_blu_ray,2,2,1.0
18,construcao_ferramentas_jardim,6,6,1.0
17,construcao_ferramentas_ferramentas,2,2,1.0
15,climatizacao,11,11,1.0
30,fashion_roupa_infanto_juvenil,1,1,1.0
14,cine_foto,2,2,1.0


In [10]:
review_satisfaction = review_satisfaction.sort_values('amount', ascending=False)
review_satisfaction.head(10)

,product_category_name,amount,total,average_satisfaction
25,esporte_lazer,170,216,0.787037
55,utilidades_domesticas,162,203,0.798030
11,cama_mesa_banho,155,207,0.748792
9,beleza_saude,142,189,0.751323
41,moveis_decoracao,133,181,0.734807
34,informatica_acessorios,120,163,0.736196
53,telefonia,116,148,0.783784
10,brinquedos,101,117,0.863248
19,cool_stuff,96,131,0.732824
33,ferramentas_jardim,74,103,0.718447


## Good Reviews by City

In [11]:
review_satisfaction_by_city = review_satisfaction_by_city.groupby(["product_category_name"]).size().reset_index(name="amount")
review_satisfaction_by_city = review_satisfaction_by_city.merge(s_review_date, on='product_category_name')
review_satisfaction_by_city['average_satisfaction'] = review_satisfaction_by_city['amount']/review_satisfaction_by_city['total']
review_satisfaction_by_city = review_satisfaction_by_city.sort_values('average_satisfaction', ascending=False)
review_satisfaction_by_city.head(10)

,product_category_name,amount,total,average_satisfaction
0,agro_industria_e_comercio,1,1,1.000000
3,artes,1,1,1.000000
2,alimentos_bebidas,3,4,0.750000
23,fashion_underwear_e_moda_praia,3,5,0.600000
21,fashion_roupa_feminina,1,2,0.500000
14,dvds_blu_ray,1,2,0.500000
12,construcao_ferramentas_ferramentas,1,2,0.500000
31,moveis_quarto,3,7,0.428571
4,audio,3,11,0.272727
22,fashion_roupa_masculina,1,4,0.250000


In [12]:
review_satisfaction_by_city = review_satisfaction_by_city.sort_values('amount', ascending=False)
review_satisfaction_by_city.head(10)

,product_category_name,amount,total,average_satisfaction
9,cama_mesa_banho,26,207,0.125604
7,beleza_saude,22,189,0.116402
39,utilidades_domesticas,20,203,0.098522
37,telefonia,20,148,0.135135
29,moveis_decoracao,19,181,0.104972
18,esporte_lazer,17,216,0.078704
13,cool_stuff,13,131,0.099237
8,brinquedos,13,117,0.111111
25,informatica_acessorios,13,163,0.079755
33,perfumaria,13,89,0.146067


## Good reviews by State

In [13]:
review_satisfaction_by_state = review_satisfaction_by_state.groupby(["product_category_name"]).size().reset_index(name="amount")
review_satisfaction_by_state = review_satisfaction_by_state.merge(s_review_date, on='product_category_name')
review_satisfaction_by_state['average_satisfaction'] = review_satisfaction_by_state['amount']/review_satisfaction_by_state['total']
review_satisfaction_by_state = review_satisfaction_by_state.sort_values('average_satisfaction', ascending=False)
review_satisfaction_by_state.head(10)

,product_category_name,amount,total,average_satisfaction
0,agro_industria_e_comercio,1,1,1.00
3,artes,1,1,1.00
14,construcao_ferramentas_ferramentas,2,2,1.00
2,alimentos_bebidas,3,4,0.75
26,fashion_roupa_masculina,3,4,0.75
27,fashion_underwear_e_moda_praia,3,5,0.60
17,dvds_blu_ray,1,2,0.50
25,fashion_roupa_feminina,1,2,0.50
11,cine_foto,1,2,0.50
1,alimentos,2,4,0.50


In [14]:
review_satisfaction_by_state = review_satisfaction_by_state.sort_values('amount', ascending=False)
review_satisfaction_by_state.head(10)

,product_category_name,amount,total,average_satisfaction
46,utilidades_domesticas,70,203,0.344828
9,cama_mesa_banho,67,207,0.323671
7,beleza_saude,61,189,0.322751
22,esporte_lazer,58,216,0.268519
35,moveis_decoracao,49,181,0.270718
44,telefonia,47,148,0.317568
29,informatica_acessorios,47,163,0.288344
8,brinquedos,34,117,0.290598
16,cool_stuff,32,131,0.244275
40,perfumaria,27,89,0.303371


# Without Considering Date

## General Good Reviews

In [15]:
total_satisfaction = satisfaction.loc[(satisfaction['review_score'] >=4)]

total_satisfaction = total_satisfaction.groupby(["product_category_name"]).size().reset_index(name="amount")
total_satisfaction = total_satisfaction.merge(s, on='product_category_name')
total_satisfaction['average_satisfaction'] = total_satisfaction['amount']/total_satisfaction['total']
total_satisfaction = total_satisfaction.sort_values('average_satisfaction', ascending=False)
total_satisfaction.head(10)

,product_category_name,amount,total,average_satisfaction
17,cds_dvds_musicais,13,14,0.928571
22,construcao_ferramentas_ferramentas,91,103,0.883495
48,livros_interesse_geral,485,553,0.877034
37,fashion_roupa_infanto_juvenil,7,8,0.875000
47,livros_importados,51,60,0.850000
49,livros_tecnicos,227,269,0.843866
64,portateis_casa_forno_e_cafe,64,76,0.842105
35,fashion_esporte,26,31,0.838710
50,malas_acessorios,913,1092,0.836081
41,flores,27,33,0.818182


In [16]:
total_satisfaction = total_satisfaction.sort_values('amount', ascending=False)
total_satisfaction.head(10)

,product_category_name,amount,total,average_satisfaction
13,cama_mesa_banho,7932,11272,0.703691
11,beleza_saude,7579,9728,0.779091
32,esporte_lazer,6752,8701,0.776003
54,moveis_decoracao,5975,8416,0.709957
44,informatica_acessorios,5755,7895,0.728942
72,utilidades_domesticas,5268,6989,0.753756
66,relogios_presentes,4459,6001,0.743043
70,telefonia,3307,4550,0.726813
40,ferramentas_jardim,3293,4361,0.755102
8,automotivo,3253,4256,0.764333


## Good Reviews by City

In [17]:
order_satisfaction_just_by_city = order_satisfaction_just_by_city.groupby(["product_category_name"]).size().reset_index(name="amount")
order_satisfaction_just_by_city = order_satisfaction_just_by_city.merge(s, on='product_category_name')
order_satisfaction_just_by_city['average_satisfaction'] = order_satisfaction_just_by_city['amount']/order_satisfaction_just_by_city['total']
order_satisfaction_just_by_city = order_satisfaction_just_by_city.sort_values('average_satisfaction', ascending=False)
order_satisfaction_just_by_city.head(10)

,product_category_name,amount,total,average_satisfaction
37,fashion_roupa_infanto_juvenil,4,8,0.500000
46,livros_importados,18,60,0.300000
17,cds_dvds_musicais,4,14,0.285714
55,moveis_quarto,26,110,0.236364
2,alimentos_bebidas,65,280,0.232143
62,portateis_casa_forno_e_cafe,16,76,0.210526
36,fashion_roupa_feminina,10,50,0.200000
1,alimentos,102,510,0.200000
15,casa_conforto_2,6,30,0.200000
51,moveis_colchao_e_estofado,7,38,0.184211


In [18]:
order_satisfaction_just_by_city = order_satisfaction_just_by_city.sort_values('amount', ascending=False)
order_satisfaction_just_by_city.head(10)

,product_category_name,amount,total,average_satisfaction
13,cama_mesa_banho,1466,11272,0.130057
11,beleza_saude,1370,9728,0.140831
32,esporte_lazer,1142,8701,0.131249
69,utilidades_domesticas,1049,6989,0.150093
44,informatica_acessorios,935,7895,0.118429
53,moveis_decoracao,929,8416,0.110385
64,relogios_presentes,665,6001,0.110815
12,brinquedos,511,4125,0.123879
67,telefonia,463,4550,0.101758
60,perfumaria,442,3435,0.128675


## Good Reviews by State

In [19]:
order_satisfaction_just_by_state = order_satisfaction_just_by_state.groupby(["product_category_name"]).size().reset_index(name="amount")
order_satisfaction_just_by_state = order_satisfaction_just_by_state.merge(s, on='product_category_name')
order_satisfaction_just_by_state['average_satisfaction'] = order_satisfaction_just_by_state['amount']/order_satisfaction_just_by_state['total']
order_satisfaction_just_by_state = order_satisfaction_just_by_state.sort_values('average_satisfaction', ascending=False)
order_satisfaction_just_by_state.head(10)

,product_category_name,amount,total,average_satisfaction
47,livros_importados,32,60,0.533333
37,fashion_roupa_infanto_juvenil,4,8,0.500000
4,artes_e_artesanato,11,24,0.458333
1,alimentos,233,510,0.456863
17,cds_dvds_musicais,6,14,0.428571
10,bebidas,163,383,0.425587
2,alimentos_bebidas,119,280,0.425000
52,moveis_colchao_e_estofado,16,38,0.421053
36,fashion_roupa_feminina,20,50,0.400000
56,moveis_quarto,43,110,0.390909


In [20]:
order_satisfaction_just_by_state = order_satisfaction_just_by_state.sort_values('amount', ascending=False)
order_satisfaction_just_by_state.head(10)

,product_category_name,amount,total,average_satisfaction
13,cama_mesa_banho,3933,11272,0.348918
11,beleza_saude,3371,9728,0.346525
32,esporte_lazer,2940,8701,0.337892
54,moveis_decoracao,2648,8416,0.314639
71,utilidades_domesticas,2584,6989,0.369724
44,informatica_acessorios,2389,7895,0.302597
66,relogios_presentes,1773,6001,0.295451
8,automotivo,1406,4256,0.330357
12,brinquedos,1359,4125,0.329455
40,ferramentas_jardim,1248,4361,0.286173
